In [1]:
import process_data
import NFL_eval
from NFL_eval import NFLMultipleModelsEval
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import sys
import os

In [23]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [3]:
interpolated_dir = os.path.join(parent_dir, "dataset_interpolated")

In [4]:
features = ["home_win", "relative_strength", "homeScore", "awayScore", "scoringPlay", "start.down", "start.distance", "start.yardLine", "end.down", "end.distance", "end.yardLine"]
eval = NFLMultipleModelsEval(interpolated_dir, features)

In [5]:
eval.process([2023, 2024])

Loading data for 2022
Loading data for .DS_Store
Loading data for 2019
Loading data for 2021
Loading data for 2020
Loading data for 2018
Done processing


In [6]:
import torch.nn as nn
import torch.nn.functional as F

# Define the feedforward neural network
class NN(nn.Module):
    def __init__(self, input_size=11, hidden_sizes=[64, 32], output_size=2):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.output = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.output(x)  # no sigmoid or softmax here

In [7]:
class NNModel:
    def __init__(self, epochs = 40, input_size=11, hidden_sizes=[64, 32], output_size=2):
        self.model = NN(input_size, hidden_sizes, output_size)
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.epochs = epochs
    def fit(self, X, y):
        for _ in range(self.epochs):
            self.optimizer.zero_grad()
            output = self.model(X)
            loss = self.criterion(output, y)
            loss.backward()
            self.optimizer.step()
    def score(self, X_test, y_test):
        self.model.eval()
        with torch.no_grad():
            logits = self.model(X_test)               # [N, 2]
            probs = F.softmax(logits, dim=1)     # [N, 2]
            y_pred = probs[:, 1]                 # Probabilities for class 1
            ss_res = torch.sum((y_test - y_pred) ** 2)
            ss_tot = torch.sum((y_test - torch.mean(y_test)) ** 2)
            r2 = 1 - ss_res / ss_tot
            return r2.item()
    def predict_proba(self, X):
        return self.model(X)


In [14]:
import importlib
importlib.reload(process_data)
importlib.reload(NFL_eval)

<module 'NFL_eval' from '/Users/aly/Documents/University of Waterloo/Winter 2025/Research/code/evalRTPF/R/NFL/ML/notebooks/NFL_eval.py'>

In [8]:
eval.train_DL(NNModel, epochs=40)

HELLO!
Training for timestep 0.0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x10 and 11x64)

In [ ]:
eval.evaluate()